In [ ]:
import json
import os
import re
import warnings
import logging

import otbApplication

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, Ellipse, Patch
#plt.rcParams["font.family"] = "DejaVu Serif"
plt.rcParams.update({'text.usetex': True, "font.family": "sans-serif", 'font.size': 18})
plt.style.use("tableau-colorblind10")  # [#006BA4, #FF800E, #BABAB, #595959, #5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF]
plt.rcParams['axes.axisbelow'] = True

import numpy as np
from numba import njit, prange
import pandas as pd
from PIL import Image
import rasterio
import xarray as xr
from scipy.ndimage import convolve

from rasterio import warp
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
def crop_img(path):
    img = np.array(Image.open(path))
    img_sum = img.sum(axis=2)
    top = 0
    bottom = img.shape[0] - 1
    left = 0
    right = img.shape[1] - 1
    for k in range(img.shape[0]):
        if (img_sum[k, :] != (255*4)).any():
            top = k
            break
    for k in range(img.shape[0]):
        if (img_sum[img.shape[0] - 1 - k, :] != (255*4)).any():
            bottom = img.shape[0] - 1 - k
            break
    for k in range(img.shape[1]):
        if (img_sum[:, k] != (255*4)).any():
            left = k
            break
    for k in range(img.shape[1]):
        if (img_sum[:, img.shape[1] - 1 - k] != (255*4)).any():
            right = img.shape[1] - 1 - k
            break
    img = img[top:bottom+1, left:right+1]
    Image.fromarray(img).save(path)
    
def custom_grid(ax, left, right, top, bottom):
    for x in range(left, right+1):
        ax.axvline(x, linewidth=1, alpha=0.3, color=gray)
    for y in range(top, bottom+1):
        ax.axhline(y, linewidth=1, alpha=0.3, color=gray)
        
def normalize_for_imshow(img: np.ndarray, q_min=0.01, q_max=0.99):
    img = np.moveaxis(img, (0,1,2), (2,0,1))
    img_normalized = np.zeros(img.shape, dtype=img.dtype)
    min_band, max_band = np.nanquantile(img, [q_min, q_max])
    for band in range(img.shape[-1]):
        img_normalized[:,:,band] = np.clip(img[:,:,band], min_band, max_band)
        img_normalized[:,:,band] = (img_normalized[:,:,band] - np.nanmin(img_normalized[:,:,band])) /\
        (np.nanmax(img_normalized[:,:,band]) - np.nanmin(img_normalized[:,:,band]))
    return np.round(img_normalized*255).astype(np.uint8)

In [ ]:
def compute_slope(data, xres=0.5, yres=0.5):
    conv_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    conv_y = conv_x.transpose()

    # Now we do the convolutions :
    gx = convolve(data, conv_x, mode="reflect")
    gy = convolve(data, conv_y, mode="reflect")

    # And eventually we do compute tan(slope) and aspect
    tan_slope = np.sqrt((gx / xres) ** 2 + (gy / yres) ** 2) / 8
    slope = np.arctan(tan_slope)

    slope = (slope * 180) / np.pi
    return slope

In [ ]:
out_path = "/work/CAMPUS/users/malinoro/outputs/These/IMG_DSM/"

blue="#006BA4"
orange="#FF800E"
gray="#ABABAB"
dark_gray="#595959"
gray_blue="#5F9ED1"
brown="#C85200"
third_gray="#898989"
light_blue="#A2C8EC"
light_orange="#FFBC79"
light_gray="#CFCFCF"

# List scenes

In [ ]:
cars_path = "/work/CAMPUS/etudes/3D/Development/malinoro/CARS_output/CO3D_performance_map"
dem_gt_path = "/work/CAMPUS/etudes/3D/Development/malinoro/Demcompare_outputs"

In [ ]:
scenes = ['Bordeaux',
          'Toulouse',
          'Pic_du_midi',
          'Peyto',
          'Monaco',
          'Montpellier',
          'Grenoble',
          'Graasubreen',
          'Hellmem',
          'Langfjordjokelen',
          'Paris']


dict_transfo = {"Bordeaux": {"drow": 0, "dcol": 1},
                "Pic_du_midi": {"drow": 3, "dcol": 4},
                "Monaco": {"drow": 11, "dcol": 12},
                "Grenoble": {"drow": 11, "dcol": 6},
                "Graasubreen": {"drow": 3, "dcol": 0},
                "Hellmem": {"drow": 7, "dcol": 0},
                "Langfjordjokelen": {"drow": 1, "dcol": 0},
                "Montpellier": {"drow": 7, "dcol": 4},
                "Paris": {"drow": 0, "dcol": 11},
                "Toulouse": {"drow": 7, "dcol": 7},
                "Peyto": {"drow": 2, "dcol": 0},
               }

dict_manual = {"Bordeaux": {"drow": 0-8, "dcol": 1+2},
               "Pic_du_midi": {"drow": 3, "dcol": 4},
               "Monaco": {"drow": 11, "dcol": 12},
               "Grenoble": {"drow": 11, "dcol": 6},
               "Graasubreen": {"drow": 10+3, "dcol": 0},
               "Hellmem": {"drow": 7+6, "dcol": 0},
               "Langfjordjokelen": {"drow": 1, "dcol": 0},
               "Montpellier": {"drow": 7+5, "dcol": 4+2},
               "Paris": {"drow": 0-1, "dcol": 11+4},
               "Toulouse": {"drow": 7+1, "dcol": 7+1},
               "Peyto": {"drow": 2, "dcol": 0},
              }

subfolder = {"Bordeaux": "",
             "Pic_du_midi": "",
             "Monaco": "",
             "Grenoble": "",
             "Graasubreen": "",
             "Hellmem": "",
             "Langfjordjokelen": "",
             "Montpellier": "Single_pair/2021-10-17_10h53m425_2021-10-17_10h53m298",
             "Paris": "Single_pair/2023-05-31_10h55m516_2023-05-31_10h55m436",
             "Toulouse": "Single_pair/10h49m208_10h49m399",
             "Peyto": "",
               }

watermask_dict = {"Bordeaux": True,
                  "Pic_du_midi": True,
                  "Monaco": True,
                  "Grenoble": False,
                  "Graasubreen": False,
                  "Hellmem": False,
                  "Langfjordjokelen": False,
                  "Montpellier": True,
                  "Paris": True,
                  "Toulouse": True,
                  "Peyto": False,
                 }

# Extract ROI

In [ ]:
"""for scene, pair in zip(["Peyto"], ["ROI_memory"]):
    for tif in ["dsm.tif", "clr.tif", "confidence_from_ambiguity_amb.tif", "dsm_inf.tif", "dsm_sup.tif"]:
        app = otbApplication.Registry.CreateApplication("ExtractROI")

        app.SetParameterString("in", os.path.join(cars_path, scene, pair, tif))
        app.SetParameterString("mode","fit")
        app.SetParameterString("mode.fit.im", os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_SEC.tif"))
        app.SetParameterString("out", os.path.join(dem_gt_path, scene, "coregistration", tif))

        app.ExecuteAndWriteOutput()
"""
#  otbcli_ExtractROI -in clr.tif -out ROI_memory/clr.tif -mode extent -mode.extent.uly 23000 -mode.extent.ulx 13000 -mode.extent.lry 33000 -mode.extent.lrx 23000

# Miniatures

In [ ]:
for scene in ['Bordeaux', 'Toulouse', 'Pic_du_midi', 'Peyto', 'Paris', 'Monaco', 'Montpellier', 'Grenoble', 'Graasubreen', 'Hellmem', 'Langfjordjokelen']:
    print(scene)
    DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
    dsm = DSM.read(1)
    bounds = DSM.bounds
    
    if scene in ['Graasubreen', 'Hellmem']:
        name_scene = "Jotunheinem"
    elif scene == "Langfjordjokelen":
        name_scene = "Langfjordjøkelen"
    else:
        name_scene = scene
    color_name = [k for k in os.listdir(os.path.join("/work/CAMPUS/etudes/3D/Development/malinoro/Pleiades", name_scene, "ortho_image")) if ".TIF"==k[-4:]][0]
    COLOR = rasterio.open(os.path.join("/work/CAMPUS/etudes/3D/Development/malinoro/Pleiades", name_scene, "ortho_image", color_name))

    bounds_warp = warp.transform_bounds(DSM.crs, COLOR.crs, bounds.left, bounds.bottom, bounds.right, bounds.top) 

    color = COLOR.read([1,2,3], window=rasterio.windows.from_bounds(*bounds_warp, COLOR.transform))

    clr_normalized = normalize_for_imshow(color)

    subsampling = min(np.ceil(clr_normalized.shape[0]/5000).astype(int),np.ceil(clr_normalized.shape[1]/5000).astype(int))
    
    GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
    gt = GT.read(1)
    
    fig, ax = plt.subplots(1, 1, figsize=(7,7))

    ax.imshow(clr_normalized[::subsampling,::subsampling, :], interpolation="nearest", aspect=clr_normalized.shape[1]/clr_normalized.shape[0]*gt.shape[0]/gt.shape[1])

    scalebar = ScaleBar(subsampling*0.5, location="lower left", pad=0.1, font_properties={"math_fontfamily": "dejavuserif"})
    ax.add_artist(scalebar)
    ax.axis("off")
    plt.savefig(os.path.join(out_path, f"miniature_{scene}.png"), dpi=100, bbox_inches='tight')
    crop_img(os.path.join(out_path, f"miniature_{scene}.png"))
    
    

    subsampling = min(np.ceil(gt.shape[0]/5000).astype(int),np.ceil(gt.shape[1]/5000).astype(int))
    fig, ax = plt.subplots(1, 1, figsize=(7,7))
    ax.imshow(gt[::subsampling,::subsampling], cmap="gray")
    ax.axis("off")
    plt.savefig(os.path.join(out_path, f"miniature_{scene}_gt.png"), dpi=100, bbox_inches='tight')
    crop_img(os.path.join(out_path, f"miniature_{scene}_gt.png"))

In [ ]:
scene = "Hellmem"
DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
dsm = DSM.read(1)
bounds = DSM.bounds

if scene in ['Graasubreen', 'Hellmem']:
    name_scene = "Jotunheinem"
elif scene == "Langfjordjokelen":
    name_scene = "Langfjordjøkelen"
else:
    name_scene = scene
color_name = [k for k in os.listdir(os.path.join("/work/CAMPUS/etudes/3D/Development/malinoro/Pleiades", name_scene, "ortho_image")) if ".TIF"==k[-4:]][0]
COLOR = rasterio.open(os.path.join("/work/CAMPUS/etudes/3D/Development/malinoro/Pleiades", name_scene, "ortho_image", color_name))

bounds_warp = warp.transform_bounds(DSM.crs, COLOR.crs, bounds.left, bounds.bottom, bounds.right, bounds.top) 

color = COLOR.read([1,2,3], window=rasterio.windows.from_bounds(*bounds_warp, COLOR.transform))

In [ ]:
color = color[:, ::3, ::6]

In [ ]:
clr_normalized = normalize_for_imshow(color)

In [ ]:
GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
gt = GT.read(1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,7))

ax.imshow(clr_normalized, interpolation="nearest", aspect=clr_normalized.shape[1]/clr_normalized.shape[0]*gt.shape[0]/gt.shape[1])

scalebar = ScaleBar(subsampling*0.5, location="lower left", pad=0.1, font_properties={"math_fontfamily": "dejavuserif"})
ax.add_artist(scalebar)
ax.axis("off")
plt.savefig(os.path.join(out_path, f"miniature_{scene}.png"), dpi=100, bbox_inches='tight')
crop_img(os.path.join(out_path, f"miniature_{scene}.png"))



subsampling = min(np.ceil(gt.shape[0]/5000).astype(int),np.ceil(gt.shape[1]/5000).astype(int))
fig, ax = plt.subplots(1, 1, figsize=(7,7))
ax.imshow(gt[::subsampling,::subsampling], cmap="gray")
ax.axis("off")
plt.savefig(os.path.join(out_path, f"miniature_gt_{scene}.png"), dpi=100, bbox_inches='tight')
crop_img(os.path.join(out_path, f"miniature_gt_{scene}.png"))

### Watermask

In [ ]:
scene = "Paris"

from numpy.lib.stride_tricks import sliding_window_view

WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
watermask = WATERMASK.read(1)
watermask[watermask==255] = 0

watermask_view = sliding_window_view(watermask, [10, 10])
img_watermask = - np.nanmax(watermask_view, axis=(2,3))

fig, ax = plt.subplots(1, 1, figsize=(7,7))

ax.imshow(img_watermask, vmin=-1, vmax=0, cmap="bwr")
ax.axis("off")
plt.savefig(os.path.join(out_path, f"watermask_{scene}.png"), dpi=100, bbox_inches='tight')
crop_img(os.path.join(out_path, f"watermask_{scene}.png"))

# Statistics

## Metrics

In [ ]:
df = pd.DataFrame(index=scenes, columns=["acc", "eps", "o_rel", "s_rel", "altimetric_bias", "r_alt", "invalid_data"])

In [ ]:
for scene in scenes:
    print(scene)
    dcol, drow = dict_manual[scene]["dcol"], dict_manual[scene]["drow"]

    DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
    dsm = DSM.read(1)

    GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
    gt = GT.read(1)

    DSM_INF = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_inf.tif"))
    dsm_inf = DSM_INF.read(1)

    DSM_SUP = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_sup.tif"))
    dsm_sup = DSM_SUP.read(1)

    gt = GT.read(1)
    gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                      (max(0, dcol), max(-dcol, 0))),
                constant_values=np.nan)

    gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
            max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]

    no_data = np.isnan(dsm)     | (dsm==DSM.nodata) | \
              np.isnan(gt)      | (gt==GT.nodata)  | \
              np.isnan(dsm_inf) | (dsm_inf==DSM_INF.nodata) |\
              np.isnan(dsm_sup) | (dsm_sup==DSM_SUP.nodata)

    with open(os.path.join(cars_path, scene, subfolder[scene], "content.json")) as f:
        content = json.load(f)["applications"]
        r_alt = float(content[list(content.keys())[0]]["grid_generation_run"]["disp_to_alt_ratio"])

    if watermask_dict[scene]:
        print("Using a watermask")
        WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
        watermask = WATERMASK.read(1)
        no_data |= (watermask>0)

    dsm[no_data] = np.nan
    dsm_inf[no_data] = np.nan
    dsm_sup[no_data] = np.nan
    gt[no_data] = np.nan
    valid_data = ~no_data

    altimetric_bias = np.nanmedian(dsm -gt)

    print(f"Altimetric bias: {altimetric_bias:.2f}m")
    gt += altimetric_bias

    print(f"Percentage of invalid data: {100*no_data.sum() / no_data.size:.0f}%")

    df.loc[scene, ["altimetric_bias", "r_alt", "invalid_data"]] = altimetric_bias, r_alt, no_data.sum() / no_data.size    
    
    correct_intervals = (gt <= dsm_sup) & (gt>=dsm_inf)

    print(f"Disp to alt ratio: {r_alt:.3f} m/disp")
    print(f"Minimum interval size: {np.nanmin(((dsm_sup-dsm_inf)[valid_data & correct_intervals]))}")
    acc = (correct_intervals)[valid_data].sum() / valid_data.sum()
    eps = np.nanmedian(np.minimum(np.abs(dsm_sup-gt)[valid_data & (~correct_intervals)], np.abs(gt-dsm_inf)[valid_data & (~correct_intervals)])) / r_alt
    s_rel = 1/r_alt*np.nanmedian((dsm_sup-dsm_inf)[valid_data])
    o_rel = 1 - np.nanmedian(np.maximum(np.abs(gt-dsm)[valid_data & correct_intervals], r_alt)/((dsm_sup-dsm_inf)[valid_data & correct_intervals]))
    
    
    df.loc[scene, ["acc", "eps", "o_rel", "s_rel"]] = acc, eps, o_rel, s_rel
    print()

In [ ]:
df.sort_index(inplace=True)
df.to_csv("DSM_metrics.csv")

In [ ]:
table = r"""\begin{table}[ht]
    \centering
    \begin{tabular}{|c||c|c|c|c|c|c|}
        \hline
        Scene & $Z_{acc}$ & $Z_\epsilon$ (pix) & $Z_{size}$ (pix) & $Z_{o}$ & $r_{alt}$ (m/pix) & invalid
        \\\hline\hline"""
for scene in df.index:
    table += f"\n        {scene.replace('_', ' ')} & " + f"{df.loc[scene, 'acc']*100:.1f}"+r"$\%$ & " + f"{df.loc[scene, 'eps']:.2f} & " f"{df.loc[scene, 's_rel']:.2f} & " + f"{df.loc[scene, 'o_rel']*100:.1f}" + r"$\%$ & " + f"{df.loc[scene, 'r_alt']:.2f}  & {df.loc[scene, 'invalid_data']*100:.1f}"+r"$\%$"
    table += r"\\\hline"
table += "\n"+r"""    \end{tabular}
    \caption{Elevation metrics for the different stereo pairs.}
    \label{tab:elevation_metrics_global}
\end{table}"""
print(table)

## Metrics per slope

In [ ]:
slope_masks_list = [[0, 2.5],
               [2.5, 5],
               [5, 10],
               [10, 15],
               [15, 20],
               [20, 30],
               [30, 40],
               [40, 50],
               [50, 70],
               [70, 90],
              ]
columns_slope = [f"{k[0]}-{k[1]}" for k in slope_masks_list]

In [ ]:
metrics = ["acc", "eps", "s_rel", "invalid_data"]
iterables = [columns_slope, metrics]

columns = pd.MultiIndex.from_product(iterables, names=["slope", "metrics"])
df = pd.DataFrame(index=scenes, columns=columns)

In [ ]:
for scene in scenes:
    print(scene)
    dcol, drow = dict_manual[scene]["dcol"], dict_manual[scene]["drow"]

    DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
    dsm = DSM.read(1)

    GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
    gt = GT.read(1)

    DSM_INF = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_inf.tif"))
    dsm_inf = DSM_INF.read(1)

    DSM_SUP = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_sup.tif"))
    dsm_sup = DSM_SUP.read(1)

    gt = GT.read(1)
    gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                      (max(0, dcol), max(-dcol, 0))),
                constant_values=np.nan)

    gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
            max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]

    no_data = np.isnan(dsm)     | (dsm==DSM.nodata) | \
              np.isnan(gt)      | (gt==GT.nodata)  | \
              np.isnan(dsm_inf) | (dsm_inf==DSM_INF.nodata) |\
              np.isnan(dsm_sup) | (dsm_sup==DSM_SUP.nodata)

    with open(os.path.join(cars_path, scene, subfolder[scene], "content.json")) as f:
        content = json.load(f)["applications"]
        r_alt = float(content[list(content.keys())[0]]["grid_generation_run"]["disp_to_alt_ratio"])

    if watermask_dict[scene]:
        WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
        watermask = WATERMASK.read(1)
        no_data |= (watermask>0)

    dsm[no_data] = np.nan
    dsm_inf[no_data] = np.nan
    dsm_sup[no_data] = np.nan
    gt[no_data] = np.nan
    valid_data = ~no_data

    altimetric_bias = np.nanmedian(dsm -gt)

    gt += altimetric_bias
    
    slope = compute_slope(gt)
    for s_mask, slope_name in zip(slope_masks_list, columns_slope):
        
        slope_mask = (s_mask[0] <= slope) & (slope < s_mask[1])
        
        correct_intervals = (gt <= dsm_sup) & (gt>=dsm_inf)
        valid_data = ~no_data
        invalid_data = (no_data[slope_mask]).sum() / (no_data[slope_mask].size)
        
        acc = (correct_intervals)[valid_data & slope_mask].sum() / (valid_data & slope_mask).sum()
        eps = np.nanmedian(np.minimum(np.abs(dsm_sup-gt)[valid_data & slope_mask & (~correct_intervals)], np.abs(gt-dsm_inf)[valid_data & slope_mask & (~correct_intervals)])) / r_alt
        s_rel = 1/r_alt*np.nanmedian((dsm_sup-dsm_inf)[valid_data & slope_mask])
    
        df.loc[scene, (slope_name, ["acc", "eps", "s_rel", "invalid_data"])] = acc, eps, s_rel, invalid_data

In [ ]:
df.sort_index(inplace=True)
df.to_csv("DSM_metrics_slope.csv")

In [ ]:
slope_edges = np.hstack((np.array(slope_masks_list)[:,0].ravel(), np.array(slope_masks_list)[-1,1]))
mlabels = [r"\(Z_{acc}\)", r"\(Z_{\varepsilon}\)", r"\(Z_{size}\)", r"\(\mathrm{Invalid~data}\)"]

for i, (metric, mlabel) in enumerate(zip(metrics, mlabels)):
    fig, ax = plt.subplots(1, 1,figsize=(4, 2))

    for scene in scenes:
        ax.stairs(df.loc[scene, (columns_slope, metric)].values.astype(float), slope_edges, baseline=None, label=scene)
    ax.set_ylabel(mlabels[i])
    ax.set_xticks([0, 10, 20, 30, 40, 50, 70, 90])
    ax.set_xlabel(r"\(\mathrm{Slope}\)")
    if metric == "acc":
        ax.set_ylim([0.5, 1.05])
    
    plt.savefig(os.path.join(out_path, f"slope_{metric}.png"), dpi=250, bbox_inches='tight')
    crop_img(os.path.join(out_path, f"slope_{metric}.png"))

In [ ]:
slope_edges = np.hstack((np.array(slope_masks_list)[:,0].ravel(), np.array(slope_masks_list)[-1,1]))

metric = "eps"
mlabel = r"\(Z_{\varepsilon}\)"

fig, ax = plt.subplots(1, 1,figsize=(4, 2))

for scene in scenes:
    if scene != "Hellmem":
        alpha=1
    else:
        alpha = 0
    ax.stairs(df.loc[scene, (columns_slope, metric)].values.astype(float), slope_edges, baseline=None, label=scene, alpha=alpha)
ax.set_ylabel(mlabel)
ax.set_ylim([0,3])
ax.set_xticks([0, 10, 20, 30, 40, 50, 70, 90])
ax.set_xlabel(r"\(\mathrm{Slope}\)")

plt.savefig(os.path.join(out_path, f"slope_{metric}_no_Hellmem.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"slope_{metric}_no_Hellmem.png"))

In [ ]:
slope_edges = np.hstack((np.array(slope_masks_list)[:,0].ravel(), np.array(slope_masks_list)[-1,1]))

metric = "eps"
mlabel = r"\(Z_{\varepsilon}\)"

fig, ax = plt.subplots(1, 1,figsize=(4, 2))
scene = "Hellmem"
ax.stairs(df.loc[scene, (columns_slope, metric)].values.astype(float), slope_edges, baseline=None, label=scene)
ax.set_ylabel(mlabel)
ax.set_xticks([0, 10, 20, 30, 40, 50, 70, 90])
ax.set_xlabel(r"\(\mathrm{Slope}\)")

## Histograms

In [ ]:
bins_eps, edges_eps = {}, {}
bins_s_rel, edges_s_rel = {}, {}

for scene in scenes:
    dcol, drow = dict_manual[scene]["dcol"], dict_manual[scene]["drow"]

    DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
    dsm = DSM.read(1)

    GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
    gt = GT.read(1)

    DSM_INF = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_inf.tif"))
    dsm_inf = DSM_INF.read(1)

    DSM_SUP = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_sup.tif"))
    dsm_sup = DSM_SUP.read(1)

    gt = GT.read(1)
    gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                      (max(0, dcol), max(-dcol, 0))),
                constant_values=np.nan)

    gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
            max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]

    no_data = np.isnan(dsm)     | (dsm==DSM.nodata) | \
              np.isnan(gt)      | (gt==GT.nodata)  | \
              np.isnan(dsm_inf) | (dsm_inf==DSM_INF.nodata) |\
              np.isnan(dsm_sup) | (dsm_sup==DSM_SUP.nodata)

    with open(os.path.join(cars_path, scene, subfolder[scene], "content.json")) as f:
        content = json.load(f)["applications"]
        r_alt = float(content[list(content.keys())[0]]["grid_generation_run"]["disp_to_alt_ratio"])

    if watermask_dict[scene]:
        WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
        watermask = WATERMASK.read(1)
        no_data |= (watermask>0)

    dsm[no_data] = np.nan
    dsm_inf[no_data] = np.nan
    dsm_sup[no_data] = np.nan
    gt[no_data] = np.nan
    valid_data = ~no_data

    altimetric_bias = np.nanmedian(dsm -gt)

    gt += altimetric_bias
    
    correct_intervals = (gt <= dsm_sup) & (gt>=dsm_inf)

    acc = (correct_intervals)[valid_data].sum() / valid_data.sum()
    eps = np.minimum(np.abs(dsm_sup-gt)[valid_data & (~correct_intervals)], np.abs(gt-dsm_inf)[valid_data & (~correct_intervals)]) / r_alt
    s_rel = 1/r_alt*(dsm_sup-dsm_inf)[valid_data]
    o_rel = 1 - np.maximum(np.abs(gt-dsm)[valid_data & correct_intervals], r_alt)/((dsm_sup-dsm_inf)[valid_data & correct_intervals])
        
    bins_eps[scene], edges_eps[scene] = np.histogram(eps, range=(0, 25), bins=50, density=True)
    bins_s_rel[scene], edges_s_rel[scene] = np.histogram(s_rel, range=(0, 25), bins=50, density=True)
    
    print(scene)
    fig, ax = plt.subplots(1, 1, figsize=(5,3))

    ax.stairs(bins_eps[scene], edges_eps[scene])
    ax.set_xlabel(r"\(Z_{\varepsilon}\)")
    ax.set_ylabel("\(\mathrm{Density}\)")
    
    plt.show()
    
    fig, ax = plt.subplots(1, 1, figsize=(5,3))

    ax.stairs(bins_s_rel[scene], edges_s_rel[scene])
    ax.set_xlabel(r"\(Z_{size}\)")
    ax.set_ylabel("\(\mathrm{Density}\)")
    plt.show()

In [ ]:
scene = "Toulouse"

fig, ax = plt.subplots(1, 1, figsize=(5,3))

ax.stairs(bins_eps[scene], edges_eps[scene])
ax.set_xlabel(r"\(Z_{\varepsilon}\)")
ax.set_ylabel("\(\mathrm{Density}\)")

plt.savefig(os.path.join(out_path, f"histogram_elevation_eps_{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"histogram_elevation_eps_{scene}.png"))

fig, ax = plt.subplots(1, 1, figsize=(5,3))

ax.stairs(bins_s_rel[scene], edges_s_rel[scene])
ax.set_xlabel(r"\(Z_{size}\)")
ax.set_ylabel("\(\mathrm{Density}\)")

plt.savefig(os.path.join(out_path, f"histogram_elevation_s_rel_{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"histogram_elevation_s_rel_{scene}.png"))

In [ ]:
scene = "Hellmem"
fig, ax = plt.subplots(1, 1, figsize=(5,3))

ax.stairs(bins_eps[scene], edges_eps[scene])
ax.set_xlabel(r"\(Z_{\varepsilon}\)")
ax.set_ylabel("\(\mathrm{Density}\)")

plt.savefig(os.path.join(out_path, f"histogram_elevation_eps_{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"histogram_elevation_eps_{scene}.png"))

fig, ax = plt.subplots(1, 1, figsize=(5,3))

ax.stairs(bins_s_rel[scene], edges_s_rel[scene])
ax.set_xlabel(r"\(Z_{size}\)")
ax.set_ylabel("\(\mathrm{Density}\)")

plt.savefig(os.path.join(out_path, f"histogram_elevation_s_rel_{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"histogram_elevation_s_rel_{scene}.png"))

## Reference simple intervals

In [ ]:
df = pd.DataFrame(index=scenes, columns=["acc", "median", "mean"])

In [ ]:
for scene in scenes:
    print(scene)
    dcol, drow = dict_manual[scene]["dcol"], dict_manual[scene]["drow"]

    DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
    dsm = DSM.read(1)

    GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
    gt = GT.read(1)

    gt = GT.read(1)
    gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                      (max(0, dcol), max(-dcol, 0))),
                constant_values=np.nan)

    gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
            max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]

    no_data = np.isnan(dsm)     | (dsm==DSM.nodata) | \
              np.isnan(gt)      | (gt==GT.nodata)

    if watermask_dict[scene]:
        WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
        watermask = WATERMASK.read(1)
        no_data |= (watermask>0)

    dsm[no_data] = np.nan
    gt[no_data] = np.nan
    valid_data = ~no_data

    altimetric_bias = np.nanmedian(dsm -gt)

    gt += altimetric_bias
    
    with open(os.path.join(cars_path, scene, subfolder[scene], "content.json")) as f:
        content = json.load(f)["applications"]
        r_alt = float(content[list(content.keys())[0]]["grid_generation_run"]["disp_to_alt_ratio"])

    correct_intervals = (gt <= dsm+r_alt) & (gt>=dsm-r_alt)

    acc = (correct_intervals)[valid_data].sum() / valid_data.sum()
    mean_error = np.nanmean(np.abs(gt-dsm)[valid_data])
    median_error = np.nanmedian(np.abs(gt-dsm)[valid_data])
    
    df.loc[scene, ["acc", "median", "mean"]] = [acc, median_error, mean_error]

In [ ]:
df

In [ ]:
df.sort_index(inplace=True)

table = ""
for scene in df.index:
    table += f"\n{scene.replace('_', ' ')} & " + f"{df.loc[scene, 'acc']*100:.1f}"+r"$\%$ & " + f"{df.loc[scene, 'median']:.2f} & " f"{df.loc[scene, 'mean']:.2f}"
    table += r"\\\hline"

print(table)

## Nuth and Kaab

In [ ]:
scene = "Paris"

In [ ]:
acc_opt = 0
k_opt, j_opt = 0, 0

k_min, k_max = -1, -1
j_min, j_max = 4, 4
for k in range(k_min,k_max+1):
    for j in range(j_min,j_max+1):
        dcol, drow = dict_transfo[scene]["dcol"]+j, dict_transfo[scene]["drow"] + k

        DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
        dsm = DSM.read(1)

        DSM_INF = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_inf.tif"))
        dsm_inf = DSM_INF.read(1)

        DSM_SUP = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_sup.tif"))
        dsm_sup = DSM_SUP.read(1)


        GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
        gt = GT.read(1)

        gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                          (max(0, dcol), max(-dcol, 0))),
                    constant_values=np.nan)

        gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
                max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]


        no_data = np.isnan(dsm)     | (dsm==DSM.nodata)         | \
                  np.isnan(gt)      | (gt==GT.nodata)           | \
                  np.isnan(dsm_inf) | (dsm_inf==DSM_INF.nodata) | \
                  np.isnan(dsm_sup) | (dsm_sup==DSM_SUP.nodata)

        if watermask_dict[scene]:
            #print("Using a watermask")
            WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
            watermask = WATERMASK.read(1)
            no_data |= (watermask>0)

        dsm[no_data] = np.nan
        dsm_inf[no_data] = np.nan
        dsm_sup[no_data] = np.nan

        gt[no_data] = np.nan
        valid_data = ~no_data


        altimetric_bias = np.nanmedian(dsm -gt)
        gt += altimetric_bias

        correct_intervals = (gt <= dsm_sup) & (gt>=dsm_inf)
        correct_intervals[no_data] = np.nan

        acc = (correct_intervals)[valid_data].sum() / (valid_data).sum()
        if acc > acc_opt:
            acc_opt = acc
            k_opt, j_opt = k, j
        print(f"{k} | {j}  ||  acc: {acc*100:.3f}")

print(f"\n {k_opt} | {j_opt}  ||  acc_opt: {acc_opt*100:.3f}")


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10, 5))
cb = ax[0].imshow(dsm-gt, vmin=-1, vmax=1, cmap="Spectral")
fig.colorbar(cb)

ax[1].imshow(gt)


In [ ]:
top, bottom = 2000, 2500
left, right = 2000, 2500

fig, ax = plt.subplots(1, 1, figsize=(10,10))

cb = ax.imshow((dsm-gt)[top:bottom+1, left:right+1],
               vmin=-3, vmax=3, cmap="Spectral",
               extent=(left, right, bottom, top))
fig.colorbar(cb)

In [ ]:
slope = compute_slope(gt)
with rasterio.Env():
    profile = DSM.profile
    tmp = slope
    tmp[np.isnan(tmp)] = DSM.nodata
    with rasterio.open('slope.tif', 'w', **profile) as dst:
        dst.write(tmp.astype(np.float32), 1)

In [ ]:
slope_dsm = compute_slope(dsm)
with rasterio.Env():
    profile = DSM.profile
    tmp = slope_dsm
    tmp[np.isnan(tmp)] = DSM.nodata
    with rasterio.open('slope_dsm.tif', 'w', **profile) as dst:
        dst.write(tmp.astype(np.float32), 1)

# Plots

## Toulouse

In [ ]:
scene = "Toulouse"

In [ ]:
dcol, drow = dict_transfo[scene]["dcol"], dict_transfo[scene]["drow"]

DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
dsm = DSM.read(1)

GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
gt = GT.read(1)

DSM_INF = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_inf.tif"))
dsm_inf = DSM_INF.read(1)

DSM_SUP = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_sup.tif"))
dsm_sup = DSM_SUP.read(1)


CLR = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/clr.tif"))
clr = CLR.read([1,2,3,4])

gt = GT.read(1)
gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                  (max(0, dcol), max(-dcol, 0))),
            constant_values=np.nan)

gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
        max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]

no_data = np.isnan(dsm)     | (dsm==DSM.nodata) | \
          np.isnan(gt)      | (gt==GT.nodata)  | \
          np.isnan(dsm_inf) | (dsm_inf==DSM_INF.nodata) |\
          np.isnan(dsm_sup) | (dsm_sup==DSM_SUP.nodata)

with open(os.path.join(cars_path, scene, subfolder[scene], "content.json")) as f:
    content = json.load(f)["applications"]
    for key in content.keys():
        try:
            r_alt = content[key]["grid_generation_run"]["disp_to_alt_ratio"]
            print(f"Disp to alt ratio: {r_alt:.3f} m/disp")
        except KeyError:
            pass

if watermask_dict[scene]:
    print("Using a watermask")
    WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
    watermask = WATERMASK.read(1)
    no_data |= (watermask>0)

    
slope = compute_slope(gt)

ndvi = (clr[3, :, :]-clr[0, :, :])/ (clr[3, :, :]+clr[0, :, :])
clr_img = normalize_for_imshow(clr)
clr_img[np.isnan(clr_img)] = 1
clr_img = np.round(clr_img*255).astype(np.uint8)


dsm[no_data] = np.nan
dsm_inf[no_data] = np.nan
dsm_sup[no_data] = np.nan
gt[no_data] = np.nan
valid_data = ~no_data
clr[:, no_data] = np.nan


altimetric_bias = np.nanmedian(dsm -gt)
print(f"Altimetric bias: {altimetric_bias:.2f}m")
gt += altimetric_bias

print(f"Percentage of invalid data: {100*no_data.sum() / no_data.size:.0f}%")

In [ ]:
correct_intervals = (gt <= dsm_sup) & (gt>=dsm_inf)
correct_intervals[ndvi>=0.7] = True
correct_intervals[no_data] = np.nan

acc = (correct_intervals)[valid_data].sum() / (valid_data).sum()
eps = np.nanmedian(np.minimum(np.abs(dsm_sup-gt)[valid_data & (~correct_intervals)], np.abs(gt-dsm_inf)[valid_data & (~correct_intervals)])) / r_alt
s_rel = 1/r_alt*np.nanmedian((dsm_sup-dsm_inf)[valid_data])
o_rel = 1 - np.nanmedian(np.maximum(np.abs(gt-dsm)[valid_data & correct_intervals], r_alt)/(dsm_sup-dsm_inf)[valid_data & correct_intervals])

print(f"acc: {acc*100:.1f}")
print(f"eps: {eps:.1f}")
print(f"s_rel: {s_rel:.1f}")
print(f"o_rel: {o_rel*100:.1f}")

In [ ]:
with rasterio.Env():
    profile = DSM.profile
    tmp = correct_intervals
    tmp[np.isnan(tmp)] = DSM.nodata
    with rasterio.open('tmp.tif', 'w', **profile) as dst:
        dst.write(tmp.astype(np.float32), 1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(dsm[::20, ::20], extent=(0, dsm.shape[1], dsm.shape[0], 0))
axes[1].imshow(correct_intervals, extent=(0, dsm.shape[1], dsm.shape[0], 0))

In [ ]:
top, bottom = 3750, 3950
left, right = 7610, 7810

fig, axes = plt.subplots(1, 2, figsize=(10, 10))

axes[0].imshow(correct_intervals[top:bottom+1, left:right+1],
               vmin=0,
               vmax=1,
               extent=(left, right, bottom, top))

axes[1].imshow(gt[top:bottom+1, left:right+1],
               vmin=np.nanquantile(gt[top:bottom+1, left:right+1], 0.05),
               vmax=np.nanquantile(gt[top:bottom+1, left:right+1], 0.95),
               extent=(left, right, bottom, top))


In [ ]:
GT_false = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_REF.tif"))
gt_false = GT_false.read(1)
gt_false[gt_false==GT_false.nodata] = np.nan

In [ ]:
# Toulouse

top, bottom = 3750, 3950
left, right = 7610, 7810

ref_points = [(7647, 3815), (7715, 3773), (7682, 3900), (7785, 3895)]


fig, axes = plt.subplots(1, 1, figsize=(5, 5))
axes.imshow(dsm[top:bottom+1, left:right+1],
               vmin=np.nanquantile(dsm[top:bottom+1, left:right+1], 0.05),
               vmax=np.nanquantile(dsm[top:bottom+1, left:right+1], 0.95),
               extent=(left, right, bottom, top), cmap="gray")


for point in ref_points:
    axes.plot([point[0], point[0]], [point[1]-5, point[1]+5], linewidth=3, color=orange)
    axes.plot([point[0]-5, point[0]+5], [point[1], point[1]], linewidth=3, color=orange)
    axes.arrow(point[0], point[1], drow, dcol, head_width=6, head_length=6, linewidth=3, color="w", length_includes_head=True)
    axes.arrow(point[0], point[1], drow, dcol, head_width=5, head_length=5, linewidth=1, color=blue, length_includes_head=True)

axes.axis("off")

plt.savefig(os.path.join(out_path, f"coregisration_planimetric_shift_cars_toulouse.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"coregisration_planimetric_shift_cars_toulouse.png"))

fig, axes = plt.subplots(1, 1, figsize=(5, 5))
axes.imshow(gt_false[top:bottom+1, left:right+1],
               vmin=np.nanquantile(gt_false[top:bottom+1, left:right+1], 0.05),
               vmax=np.nanquantile(gt_false[top:bottom+1, left:right+1], 0.95),
               extent=(left, right, bottom, top), cmap="gray")

for point in ref_points:
    axes.plot([point[0], point[0]], [point[1]-5, point[1]+5], linewidth=3, color=orange)
    axes.plot([point[0]-5, point[0]+5], [point[1], point[1]], linewidth=3, color=orange)

axes.axis("off")
plt.savefig(os.path.join(out_path, f"coregisration_planimetric_shift_gt_toulouse.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"coregisration_planimetric_shift_gt_toulouse.png"))

In [ ]:
# Toulouse
row = 4500
col_inf, col_sup = 3400, 3800

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), gt[row, col_inf:col_sup]-altimetric_bias, color=blue, label=r"\(\mathrm{DSM_{true}}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), gt[row, col_inf:col_sup], color=blue, label=r"\(\mathrm{DSM_{true}}+\tilde{dh}\)", linestyle=":", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), dsm[row, col_inf:col_sup], color=orange, label=r"\(\mathrm{DSM}\)", linewidth=2, alpha=0.7)

ax.arrow(3490, 133, 0, altimetric_bias-5, head_width=5, head_length=4, linewidth=1, color=blue, length_includes_head=True)
ax.text(3475, 150, r"\(\tilde{dh}\)")

ax.set_ylabel("\(\mathrm{Elevation~ (m)}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

ax.legend(loc=(0.005, 0.68), framealpha=1)

plt.savefig(os.path.join(out_path, f"coregisration_altimetric_shift_toulouse.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"coregisration_altimetric_shift_toulouse.png"))

## Paris

In [ ]:
scene = "Paris"

In [ ]:
dcol, drow = dict_transfo[scene]["dcol"], dict_transfo[scene]["drow"]

DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
dsm = DSM.read(1)

GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
gt = GT.read(1)

DSM_INF = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_inf.tif"))
dsm_inf = DSM_INF.read(1)

DSM_SUP = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_sup.tif"))
dsm_sup = DSM_SUP.read(1)


CLR = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/clr.tif"))
clr = CLR.read([1,2,3,4])

gt = GT.read(1)
gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                  (max(0, dcol), max(-dcol, 0))),
            constant_values=np.nan)

gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
        max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]

no_data = np.isnan(dsm)     | (dsm==DSM.nodata) | \
          np.isnan(gt)      | (gt==GT.nodata)  | \
          np.isnan(dsm_inf) | (dsm_inf==DSM_INF.nodata) |\
          np.isnan(dsm_sup) | (dsm_sup==DSM_SUP.nodata)

with open(os.path.join(cars_path, scene, subfolder[scene], "content.json")) as f:
    content = json.load(f)["applications"]
    for key in content.keys():
        try:
            r_alt = content[key]["grid_generation_run"]["disp_to_alt_ratio"]
            print(f"Disp to alt ratio: {r_alt:.3f} m/disp")
        except KeyError:
            pass

if watermask_dict[scene]:
    print("Using a watermask")
    WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
    watermask = WATERMASK.read(1)
    no_data |= (watermask>0)

    
slope = compute_slope(gt)

ndvi = (clr[3, :, :]-clr[0, :, :])/ (clr[3, :, :]+clr[0, :, :])

dsm[no_data] = np.nan
dsm_inf[no_data] = np.nan
dsm_sup[no_data] = np.nan
gt[no_data] = np.nan
valid_data = ~no_data
clr[:, no_data] = np.nan


altimetric_bias = np.nanmedian(dsm -gt)
print(f"Altimetric bias: {altimetric_bias:.2f}m")
gt += altimetric_bias

print(f"Percentage of invalid data: {100*no_data.sum() / no_data.size:.0f}%")

In [ ]:
correct_intervals = (gt <= dsm_sup) & (gt>=dsm_inf)
correct_intervals[ndvi>=0.7] = True
correct_intervals[no_data] = np.nan

acc = (correct_intervals)[valid_data].sum() / (valid_data).sum()
eps = np.nanmedian(np.minimum(np.abs(dsm_sup-gt)[valid_data & (~correct_intervals)], np.abs(gt-dsm_inf)[valid_data & (~correct_intervals)])) / r_alt
s_rel = 1/r_alt*np.nanmedian((dsm_sup-dsm_inf)[valid_data])
o_rel = 1 - np.nanmedian(np.maximum(np.abs(gt-dsm)[valid_data & correct_intervals], r_alt)/(dsm_sup-dsm_inf)[valid_data & correct_intervals])

print(f"acc: {acc*100:.1f}")
print(f"eps: {eps:.1f}")
print(f"s_rel: {s_rel:.1f}")
print(f"o_rel: {o_rel*100:.1f}")

In [ ]:
with rasterio.Env():
    profile = DSM.profile
    tmp = correct_intervals
    tmp[np.isnan(tmp)] = DSM.nodata
    with rasterio.open('tmp.tif', 'w', **profile) as dst:
        dst.write(tmp.astype(np.float32), 1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(dsm[::20, ::20], extent=(0, dsm.shape[1], dsm.shape[0], 0))
axes[1].imshow(correct_intervals, extent=(0, dsm.shape[1], dsm.shape[0], 0))

In [ ]:
dsm.shape

In [ ]:
top, bottom = 1500, 2250
left, right = 3750, 4500

fig, axes = plt.subplots(1, 2, figsize=(10, 10))

axes[0].imshow(correct_intervals[top:bottom+1, left:right+1],
               vmin=0,
               vmax=1,
               extent=(left, right, bottom, top))

axes[1].imshow(gt[top:bottom+1, left:right+1],
               vmin=np.nanquantile(gt[top:bottom+1, left:right+1], 0.05),
               vmax=np.nanquantile(gt[top:bottom+1, left:right+1], 0.95),
               extent=(left, right, bottom, top))


In [ ]:
# Paris
row = 1875
col_inf, col_sup = 4100, 4480

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), dsm_inf[row, col_inf:col_sup], color=orange, linestyle="--", label=r"\(\underline{\mathrm{DSM}},~\overline{\mathrm{DSM}}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), dsm_sup[row, col_inf:col_sup], color=orange, linestyle="--", linewidth=2, alpha=0.7)

ax.plot(np.arange(col_inf, col_sup), dsm[row, col_inf:col_sup], color=orange, label=r"\(\mathrm{DSM}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), gt[row, col_inf:col_sup], color=blue, label=r"\(\mathrm{DSM_{true}}\)", linewidth=2.5)

ax.set_ylabel("\(\mathrm{Elevation~ (m)}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

ax.legend(framealpha=1)

plt.savefig(os.path.join(out_path, f"paris_error_tree.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"paris_error_tree.png"))

top, bottom = row-50, row+50
left, right = col_inf, col_sup

fig, ax = plt.subplots(1, 1, figsize=(10, 3))

ax.imshow(clr_img[top:bottom+1, left:right+1, 0:3],
               vmin=0,
               vmax=1,
               extent=(left, right, bottom, top))
ax.axes.get_yaxis().set_visible(False)
ax.plot([col_inf, col_sup], [row, row], color=orange)

plt.savefig(os.path.join(out_path, f"paris_error_tree_clr.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"paris_error_tree_clr.png"))

fig, ax = plt.subplots(1, 1, figsize=(10, 3))
ax.imshow(gt[top:bottom+1, left:right+1],
               vmin=np.nanquantile(gt[top:bottom+1, left:right+1], 0.05),
               vmax=np.nanquantile(gt[top:bottom+1, left:right+1], 0.95),
               extent=(left, right, bottom, top), cmap="gray")
ax.plot([col_inf, col_sup], [row, row], color=orange)

ax.axes.get_yaxis().set_visible(False)


plt.savefig(os.path.join(out_path, f"paris_error_tree_gt.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"paris_error_tree_gt.png"))

## Monaco

In [ ]:
scene = "Monaco"

In [ ]:
dcol, drow = dict_manual[scene]["dcol"], dict_manual[scene]["drow"]

DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
dsm = DSM.read(1)

GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
gt = GT.read(1)

DSM_INF = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_inf.tif"))
dsm_inf = DSM_INF.read(1)

DSM_SUP = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_sup.tif"))
dsm_sup = DSM_SUP.read(1)


CLR = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/clr.tif"))
clr = CLR.read([1,2,3,4])

gt = GT.read(1)
gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                  (max(0, dcol), max(-dcol, 0))),
            constant_values=np.nan)

gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
        max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]

custom_mask = np.zeros(dsm.shape, dtype=np.bool_)
#custom_mask[250:1500, 1180:1700] = True

no_data = np.isnan(dsm)     | (dsm==DSM.nodata) | \
          np.isnan(gt)      | (gt==GT.nodata)  | \
          np.isnan(dsm_inf) | (dsm_inf==DSM_INF.nodata) |\
          np.isnan(dsm_sup) | (dsm_sup==DSM_SUP.nodata) | custom_mask

with open(os.path.join(cars_path, scene, subfolder[scene], "content.json")) as f:
    content = json.load(f)["applications"]
    for key in content.keys():
        try:
            r_alt = content[key]["grid_generation_run"]["disp_to_alt_ratio"]
            print(f"Disp to alt ratio: {r_alt:.3f} m/disp")
        except KeyError:
            pass

if watermask_dict[scene]:
    print("Using a watermask")
    WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
    watermask = WATERMASK.read(1)
    no_data |= (watermask>0)

    
slope = compute_slope(gt)

dsm[no_data] = np.nan
dsm_inf[no_data] = np.nan
dsm_sup[no_data] = np.nan
#gt[no_data] = np.nan
valid_data = ~no_data
clr[:, no_data] = np.nan

altimetric_bias = np.nanmedian(dsm -gt)
print(f"Altimetric bias: {altimetric_bias:.2f}m")
gt += altimetric_bias

print(f"Percentage of invalid data: {100*no_data.sum() / no_data.size:.1f}%")

In [ ]:
(1500-250)*(1700-1180)/valid_data.sum()

In [ ]:
correct_intervals = (gt <= dsm_sup) & (gt>=dsm_inf)
correct_intervals[no_data] = np.nan

error = (dsm-gt)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(6, 6))
cs = axes.imshow((dsm-gt)[0::5, 0::5], extent=(0, dsm.shape[1], dsm.shape[0], 0), cmap="bwr", vmin=-3, vmax=3)
axes.axis("off")
cbar = fig.colorbar(cs, shrink=0.5)

plt.savefig(os.path.join(out_path, f"{scene}_errors.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_errors.png"))

fig, axes = plt.subplots(1, 1, figsize=(6, 6))
axes.axis("off")
axes.imshow(1-correct_intervals.astype(float), extent=(0, dsm.shape[1], dsm.shape[0], 0), cmap="bwr", vmin=-1, vmax=1)

plt.savefig(os.path.join(out_path, f"{scene}_wrong_intervals.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"{scene}_wrong_intervals.png"))

In [ ]:
from rasterio import warp
color_name = [k for k in os.listdir(os.path.join("/work/CAMPUS/etudes/3D/Development/malinoro/Pleiades", scene, "ortho_image")) if ".TIF"==k[-4:]][0]
COLOR = rasterio.open(os.path.join("/work/CAMPUS/etudes/3D/Development/malinoro/Pleiades", scene, "ortho_image", color_name))
bounds = DSM.bounds

bounds_warp = warp.transform_bounds(DSM.crs, COLOR.crs, bounds.left, bounds.bottom, bounds.right, bounds.top) 

color = COLOR.read([1,2,3], window=rasterio.windows.from_bounds(*bounds_warp, COLOR.transform))

clr_normalized = normalize_for_imshow(color)

In [ ]:
top, bottom = 0, 1750
left, right = 1000, 2000

fig, axes = plt.subplots(1, 1, figsize=(6, 6))

axes.imshow(1-correct_intervals[top:bottom+1, left:right+1],
               vmin=-1,
               vmax=1,
               extent=(left, right, bottom, top), cmap="bwr")
axes.axis("off")
plt.savefig(os.path.join(out_path,f"Carriere_wrong_intervals_{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"Carriere_wrong_intervals_{scene}.png"))

fig, axes = plt.subplots(1, 1, figsize=(6, 6))
axes.imshow(gt[top:bottom+1, left:right+1],
               vmin=np.nanquantile(gt[top:bottom+1, left:right+1], 0.05),
               vmax=np.nanquantile(gt[top:bottom+1, left:right+1], 0.95),
               extent=(left, right, bottom, top),cmap="gray")
axes.plot([1000, 2000], [800, 800], c=orange)
axes.axis("off")
plt.savefig(os.path.join(out_path, f"Carriere_gt_{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"Carriere_gt_{scene}.png"))

fig, axes = plt.subplots(1, 1, figsize=(6, 6))
crop_clr_normalized = clr_normalized[top+300:bottom+300+1, left+220:right+480+1]
axes.axis("off")
axes.imshow(crop_clr_normalized, interpolation="nearest", aspect=crop_clr_normalized.shape[1]/crop_clr_normalized.shape[0]*(bottom-top+1)/(right-left+1))
scalebar = ScaleBar(0.5, location="lower left", pad=0.1, font_properties={"math_fontfamily": "dejavuserif"})
axes.add_artist(scalebar)
plt.savefig(os.path.join(out_path, f"Carriere_RGB_{scene}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"Carriere_RGB_{scene}.png"))

In [ ]:
# Toulouse
row = 800
col_inf, col_sup = 1000, 2000

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), dsm_inf[row, col_inf:col_sup], color=orange, linestyle="--", label=r"\(\underline{\mathrm{DSM}},~\overline{\mathrm{DSM}}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), dsm_sup[row, col_inf:col_sup], color=orange, linestyle="--", linewidth=2, alpha=0.7)

ax.plot(np.arange(col_inf, col_sup), dsm[row, col_inf:col_sup], color=orange, label=r"\(\mathrm{DSM}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), gt[row, col_inf:col_sup], color=blue, label=r"\(\mathrm{DSM_{true}}\)", linewidth=2.5)

ax.set_ylabel("\(\mathrm{Elevation~ (m)}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.set_ylim([400, 525])

ax.grid(True)

ax.legend(framealpha=1)

plt.savefig(os.path.join(out_path, f"Carriere_row_{row}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"Carriere_row_{row}.png"))

## Hellmem

In [ ]:
scene = "Hellmem"

In [ ]:
dcol, drow = dict_manual[scene]["dcol"], dict_manual[scene]["drow"]

DSM = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm.tif"))
dsm = DSM.read(1)

GT = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/reproj_coreg_REF.tif"))
gt = GT.read(1)

DSM_INF = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_inf.tif"))
dsm_inf = DSM_INF.read(1)

DSM_SUP = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/dsm_sup.tif"))
dsm_sup = DSM_SUP.read(1)

gt = GT.read(1)
gt = np.pad(gt, ((max(0, drow), max(-drow, 0)),
                  (max(0, dcol), max(-dcol, 0))),
            constant_values=np.nan)

gt = gt[max(0, -drow):min(gt.shape[0]-drow, gt.shape[0]),
        max(0, -dcol):min(gt.shape[1]-dcol, gt.shape[1])]

no_data = np.isnan(dsm)     | (dsm==DSM.nodata) | \
          np.isnan(gt)      | (gt==GT.nodata)  | \
          np.isnan(dsm_inf) | (dsm_inf==DSM_INF.nodata) |\
          np.isnan(dsm_sup) | (dsm_sup==DSM_SUP.nodata)

with open(os.path.join(cars_path, scene, subfolder[scene], "content.json")) as f:
    content = json.load(f)["applications"]
    for key in content.keys():
        try:
            r_alt = content[key]["grid_generation_run"]["disp_to_alt_ratio"]
            print(f"Disp to alt ratio: {r_alt:.3f} m/disp")
        except KeyError:
            pass

if watermask_dict[scene]:
    print("Using a watermask")
    WATERMASK = rasterio.open(os.path.join(dem_gt_path, scene, "coregistration/watermask.tif"))
    watermask = WATERMASK.read(1)
    no_data |= (watermask>0)

    
slope = compute_slope(gt)

dsm[no_data] = np.nan
dsm_inf[no_data] = np.nan
dsm_sup[no_data] = np.nan
gt[no_data] = np.nan
valid_data = ~no_data


altimetric_bias = np.nanmedian(dsm -gt)
print(f"Altimetric bias: {altimetric_bias:.2f}m")
gt += altimetric_bias

print(f"Percentage of invalid data: {100*no_data.sum() / no_data.size:.1f}%")

In [ ]:
correct_intervals = (gt <= dsm_sup) & (gt>=dsm_inf)
correct_intervals[no_data] = np.nan

acc = (correct_intervals)[valid_data].sum() / (valid_data).sum()
eps = np.nanmedian(np.minimum(np.abs(dsm_sup-gt)[valid_data & (~correct_intervals)], np.abs(gt-dsm_inf)[valid_data & (~correct_intervals)])) / r_alt
s_rel = 1/r_alt*np.nanmedian((dsm_sup-dsm_inf)[valid_data])
o_rel = 1 - np.nanmedian(np.maximum(np.abs(gt-dsm)[valid_data & correct_intervals], r_alt)/(dsm_sup-dsm_inf)[valid_data & correct_intervals])

print(f"acc: {acc*100:.1f}")
print(f"eps: {eps:.2f}")
print(f"s_rel: {s_rel:.2f}")
print(f"o_rel: {o_rel*100:.1f}")

eps = np.minimum(np.abs(dsm_sup-gt), np.abs(gt-dsm_inf)) / r_alt
eps[valid_data & (correct_intervals)] = np.nan 

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 12))
axes.imshow(eps, extent=(0, dsm.shape[1], dsm.shape[0], 0))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 6))

axes[0].imshow(dsm[::20, ::20], extent=(0, dsm.shape[1], dsm.shape[0], 0))
axes[1].imshow(correct_intervals, extent=(0, dsm.shape[1], dsm.shape[0], 0))
axes[2].imshow(eps, extent=(0, dsm.shape[1], dsm.shape[0], 0))

In [ ]:
top, bottom = 0, 5000
left, right = 0, 5000

fig, axes = plt.subplots(1, 2, figsize=(10, 10))

axes[0].imshow(correct_intervals[top:bottom+1, left:right+1],
               vmin=0,
               vmax=1,
               extent=(left, right, bottom, top))

axes[1].imshow(eps[top:bottom+1, left:right+1],
               vmin=0,
               vmax=25,
               extent=(left, right, bottom, top))


In [ ]:
# Toulouse
row = 900
col_inf, col_sup = 1000, 1750

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), dsm_inf[row, col_inf:col_sup], color=orange, linestyle="--", label=r"\(\underline{\mathrm{DSM}},~\overline{\mathrm{DSM}}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), dsm_sup[row, col_inf:col_sup], color=orange, linestyle="--", linewidth=2, alpha=0.7)

ax.plot(np.arange(col_inf, col_sup), dsm[row, col_inf:col_sup], color=orange, label=r"\(\mathrm{DSM}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), gt[row, col_inf:col_sup], color=blue, label=r"\(\mathrm{DSM_{true}}\)", linewidth=2.5)

ax.set_ylabel("\(\mathrm{Elevation~ (m)}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

ax.legend(framealpha=1)